# Import Libraries

In [1]:
import os
import datetime as dt
import sun_position as sp
import shadowingfunctions as shadow
from osgeo import gdal, osr
from osgeo.gdalconst import *
import numpy as np
import geopandas as gpd
import pandas as pd

pd.set_option('display.max_columns', None) #-- to show all columns in a dataframe. To reset: pd.reset_option(“max_columns”)


# Processing whole tiles

In [2]:
### Shade calculation setup

def shadecalculation_setup(filepath_dsm = 'None', filepath_veg = 'None', date = dt.datetime.now(), intervalTime = 30, onetime =1, filepath_save='None', UTC=0, dst=1, useveg=0, trunkheight=25, transmissivity=20):
    '''Calculates spot, hourly and or daily shading for a DSM
    Needs: 
    filepath_dsm = a path to a (building) dsm,
    date = a datetime object (defaults to datetime.now()),
    intervaltime = a integer in minutes as interval,
    onetime = to differentiate between single shade cast and day (default is single timestamp)
    filepath_save = 'path to the folder in which to save the files'
    UTC = defaults to 0, optional in plugin, denotes the UCT shift (AMS +2)
    dst = defaults to 1, not sure what this does. 1 or 0
    '''
    print(date)


    gdal_dsm = gdal.Open(filepath_dsm)
    dsm = gdal_dsm.ReadAsArray().astype(float)

    # response to issue #85
    nd = gdal_dsm.GetRasterBand(1).GetNoDataValue()
    dsm[dsm == nd] = 0.
    if dsm.min() < 0:
        dsm = dsm + np.abs(dsm.min())

    sizex = dsm.shape[0]
    sizey = dsm.shape[1]

    old_cs = osr.SpatialReference()
    dsm_ref = gdal_dsm.GetProjection()
    print(dsm_ref)
    # dsm_ref = dsmlayer.crs().toWkt()
    old_cs.ImportFromWkt(dsm_ref)

    wgs84_wkt = """
    GEOGCS["WGS 84",
        DATUM["WGS_1984",
            SPHEROID["WGS 84",6378137,298.257223563,
                AUTHORITY["EPSG","7030"]],
            AUTHORITY["EPSG","6326"]],
        PRIMEM["Greenwich",0,
            AUTHORITY["EPSG","8901"]],
        UNIT["degree",0.01745329251994328,
            AUTHORITY["EPSG","9122"]],
        AUTHORITY["EPSG","4326"]]"""

    new_cs = osr.SpatialReference()
    new_cs.ImportFromWkt(wgs84_wkt)

    transform = osr.CoordinateTransformation(old_cs, new_cs)

    width = gdal_dsm.RasterXSize
    height = gdal_dsm.RasterYSize
    gt = gdal_dsm.GetGeoTransform()
    minx = gt[0]
    miny = gt[3] + width*gt[4] + height*gt[5]
    lonlat = transform.TransformPoint(minx, miny)
    geotransform = gdal_dsm.GetGeoTransform()
    scale = 1 / geotransform[1]
    
    gdalver = float(gdal.__version__[0])
    if gdalver >= 3.:
        lon = lonlat[1] #changed to gdal 3
        lat = lonlat[0] #changed to gdal 3
    else:
        lon = lonlat[0] #changed to gdal 2
        lat = lonlat[1] #changed to gdal 2
    # print('lon:' + str(lon))
    # print('lat:' + str(lat))

    ## Import vegetation dsm

    trans = transmissivity / 100.0


    if useveg == 1:
        usevegdem = 1

        # Changed import conditions
        if filepath_veg == "None":
            print("No vegetation filepath given")
            return

        # load raster
        # gdal.AllRegister()
        # provider = vegdsm.dataProvider()
        # filePathOld = str(provider.dataSourceUri())
        dataSet = gdal.Open(filepath_veg)
        vegdsm = dataSet.ReadAsArray().astype(float)

        vegsizex = vegdsm.shape[0]
        vegsizey = vegdsm.shape[1]

        if not (vegsizex == sizex) & (vegsizey == sizey):
            print("Error; All grids must be of same extent and resolution")
            return
        ## Removed the option for Trunkzone DSM. Defaults to 25%
        # if self.dlg.checkBoxTrunkExist.isChecked():
        #     vegdsm2 = self.layerComboManagerVEGDSM2.currentLayer()

        #     if vegdsm2 is None:
        #         QMessageBox.critical(None, "Error", "No valid trunk zone DSM selected")
        #         return

        #     # load raster
        #     gdal.AllRegister()
        #     provider = vegdsm2.dataProvider()
        #     filePathOld = str(provider.dataSourceUri())
        #     dataSet = gdal.Open(filePathOld)
        #     vegdsm2 = dataSet.ReadAsArray().astype(float)
        # else:
        trunkratio = trunkheight / 100.0
        vegdsm2 = vegdsm * trunkratio

        vegsizex = vegdsm2.shape[0]
        vegsizey = vegdsm2.shape[1]

        if not (vegsizex == sizex) & (vegsizey == sizey):  # &
            print("Error; All grids must be of same extent and resolution")
            return
    else:
        vegdsm = 0
        vegdsm2 = 0
        usevegdem = 0

    ## REMOVED WALLSHADOW FUNCTIONS FOR NOW.
    # if self.dlg.checkBoxWallsh.isChecked():
    #     wallsh = 1
    #     # wall height layer
    #     whlayer = self.layerComboManagerWH.currentLayer()
    #     if whlayer is None:
    #             QMessageBox.critical(None, "Error", "No valid wall height raster layer is selected")
    #             return
    #     provider = whlayer.dataProvider()
    #     filepath_wh= str(provider.dataSourceUri())
    #     self.gdal_wh = gdal.Open(filepath_wh)
    #     wheight = self.gdal_wh.ReadAsArray().astype(float)
    #     vhsizex = wheight.shape[0]
    #     vhsizey = wheight.shape[1]
    #     if not (vhsizex == sizex) & (vhsizey == sizey):  # &
    #         QMessageBox.critical(None, "Error", "All grids must be of same extent and resolution")
    #         return

    #     # wall aspectlayer
    #     walayer = self.layerComboManagerWA.currentLayer()
    #     if walayer is None:
    #             QMessageBox.critical(None, "Error", "No valid wall aspect raster layer is selected")
    #             return
    #     provider = walayer.dataProvider()
    #     filepath_wa= str(provider.dataSourceUri())
    #     self.gdal_wa = gdal.Open(filepath_wa)
    #     waspect = self.gdal_wa.ReadAsArray().astype(float)
    #     vasizex = waspect.shape[0]
    #     vasizey = waspect.shape[1]
    #     if not (vasizex == sizex) & (vasizey == sizey):
    #         QMessageBox.critical(None, "Error", "All grids must be of same extent and resolution")
    #         return
    # else:
    wallsh = 0
    wheight = 0
    waspect = 0
    
    if filepath_save == 'None':
        print("Error", "No output path given")
        return
    else:
        # date = self.dlg.calendarWidget.selectedDate()
        year = date.year
        month = date.month
        day = date.day
        # TODO: Check what UTC does. Optional in plugin, so set to 0
        # UTC = self.dlg.spinBoxUTC.value()
        if onetime == 1:
            time = date.time()
            print(time)
            hour = time.hour
            minu = time.minute
            sec = time.second
        else:
            onetime = 0
            hour = 0
            minu = 0
            sec = 0

        tv = [year, month, day, hour, minu, sec]
        # TODO: What exactly is set by timeInterval. For now this just takes the minutes as set above. 
        # intervalTime = self.dlg.intervalTimeEdit.time()
        # self.timeInterval = intervalTime.minute() + (intervalTime.hour() * 60) + (intervalTime.second()/60)

        shadowresult = dailyshading(dsm, vegdsm, vegdsm2, scale, lon, lat, sizex, sizey, tv, UTC, usevegdem,
                                    intervalTime, onetime, filepath_save, gdal_dsm, trans, 
                                    dst, wallsh, wheight, waspect)

        shfinal = shadowresult["shfinal"]
        time_vector = shadowresult["time_vector"]

        if onetime == 0:
            timestr = time_vector.strftime("%Y%m%d")
            savestr = '/shadow_fraction_on_'
        else:
            timestr = time_vector.strftime("%Y%m%d_%H%M")
            savestr = '/Shadow_at_'

    filename = filepath_save + savestr + timestr + '.tif'

## TODO: change to saverasternd or other function
    saveraster(gdal_dsm, filename, shfinal)


############## DAILYSHADING ################

def dailyshading(dsm, vegdsm, vegdsm2, scale, lon, lat, sizex, sizey, tv, UTC, usevegdem, timeInterval, onetime, folder, gdal_data, trans, dst, wallshadow, wheight, waspect):

    # lon = lonlat[0]
    # lat = lonlat[1]
    year = tv[0]
    month = tv[1]
    day = tv[2]

    alt = np.median(dsm)
    location = {'longitude': lon, 'latitude': lat, 'altitude': alt}

    if usevegdem == 1:
        psi = trans
        # amaxvalue
        vegmax = vegdsm.max()
        amaxvalue = dsm.max() - dsm.min()
        amaxvalue = np.maximum(amaxvalue, vegmax)

        # Elevation vegdsms if buildingDSM includes ground heights
        vegdem = vegdsm + dsm
        vegdem[vegdem == dsm] = 0
        vegdem2 = vegdsm2 + dsm
        vegdem2[vegdem2 == dsm] = 0

        # Bush separation
        bush = np.logical_not((vegdem2*vegdem))*vegdem

    #     vegshtot = np.zeros((sizex, sizey))
    # else:
        
    shtot = np.zeros((sizex, sizey))

    if onetime == 1:
        itera = 1
    else:
        itera = int(np.round(1440 / timeInterval))

    alt = np.zeros(itera)
    azi = np.zeros(itera)
    hour = int(0)
    index = 0
    time = dict()
    time['UTC'] = UTC

    if wallshadow == 1:
        walls = wheight
        dirwalls = waspect
    else: 
        walls = np.zeros((sizex, sizey))
        dirwalls = np.zeros((sizex, sizey))

    for i in range(0, itera):
        if onetime == 0:
            minu = int(timeInterval * i)
            if minu >= 60:
                hour = int(np.floor(minu / 60))
                minu = int(minu - hour * 60)
        else:
            minu = tv[4]
            hour = tv[3]
        
        doy = day_of_year(year, month, day)

        ut_time = doy - 1. + ((hour - dst) / 24.0) + (minu / (60. * 24.0)) + (0. / (60. * 60. * 24.0))
        
        if ut_time < 0:
            year = year - 1
            month = 12
            day = 31
            doy = day_of_year(year, month, day)
            ut_time = ut_time + doy - 1

        HHMMSS = dectime_to_timevec(ut_time)
        
        time['year'] = year
        time['month'] = month
        time['day'] = day
        time['hour'] = HHMMSS[0]
        time['min'] = HHMMSS[1]
        time['sec'] = HHMMSS[2]

        sun = sp.sun_position(time, location)
        alt[i] = 90. - sun['zenith']
        azi[i] = sun['azimuth']

        if time['sec'] == 59: #issue 228 and 256
            time['sec'] = 0
            time['min'] = time['min'] + 1
            if time['min'] == 60:
                time['min'] = 0
                time['hour'] = time['hour'] + 1
                if time['hour'] == 24:
                    time['hour'] = 0

        time_vector = dt.datetime(year, month, day, time['hour'], time['min'], time['sec'])
        timestr = time_vector.strftime("%Y%m%d_%H%M")

        if alt[i] > 0:
            if wallshadow == 1: # Include wall shadows (Issue #121)
                if usevegdem == 1:
                    vegsh, sh, _, wallsh, _, wallshve, _, _ = shadowingfunction_wallheight_23(dsm, vegdem, vegdem2,
                                                azi[i], alt[i], scale, amaxvalue, bush, walls, dirwalls * np.pi / 180.)
                    sh = sh - (1 - vegsh) * (1 - psi)
                    if onetime == 0:
                        filenamewallshve = folder + '/Facadeshadow_fromvegetation_' + timestr + '_LST.tif'
                        saveraster(gdal_data, filenamewallshve, wallshve)
                else:
                    sh, wallsh, _, _, _ = shadowingfunction_wallheight_13(dsm, azi[i], alt[i], scale,
                                                                                        walls, dirwalls * np.pi / 180.)
                    # shtot = shtot + sh
                
                if onetime == 0:
                    filename = folder + '/Shadow_ground_' + timestr + '_LST.tif'
                    saveraster(gdal_data, filename, sh)
                    filenamewallsh = folder + '/Facadeshadow_frombuilding_' + timestr + '_LST.tif'
                    saveraster(gdal_data, filenamewallsh, wallsh)
                    

            else:
                if usevegdem == 0:
                    sh = shadow.shadowingfunctionglobalradiation(dsm, azi[i], alt[i], scale, 0)
                    # shtot = shtot + sh
                else:
                    # changed to "optimized" function
                    shadowresult = shadow.shadowingfunction_20(dsm, vegdem, vegdem2, azi[i], alt[i], scale, amaxvalue, bush, 0)




                    vegsh = shadowresult["vegsh"]
                    sh = shadowresult["sh"]
                    sh=sh-(1-vegsh)*(1-psi)
                # vegshtot = vegshtot + sh
                # sh = shadow.shadowingfunctionglobalradiation(dsm, azi[i], alt[i], scale, 0)

                if onetime == 0:
                    filename = folder + '/Shadow_' + timestr + '_LST.tif'
                    ## EDITED 
                    saveraster(gdal_data, filename, sh)

            shtot = shtot + sh
            index += 1

    shfinal = shtot / index

    if wallshadow == 1:
        if onetime == 1:
            filenamewallsh = folder + '/Facadeshadow_frombuilding_' + timestr + '_LST.tif'
            saveraster(gdal_data, filenamewallsh, wallsh)
            if usevegdem == 1:
                filenamewallshve = folder + '/Facadeshadow_fromvegetation_' + timestr + '_LST.tif'
                saveraster(gdal_data, filenamewallshve, wallshve)

    shadowresult = {'shfinal': shfinal, 'time_vector': time_vector}

    # dlg.progressBar.setValue(0)

    return shadowresult


def day_of_year(yy, month, day):
    if (yy % 4) == 0:
        if (yy % 100) == 0:
            if (yy % 400) == 0:
                leapyear = 1
            else:
                leapyear = 0
        else:
            leapyear = 1
    else:
        leapyear = 0

    if leapyear == 1:
        dayspermonth = [31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
    else:
        dayspermonth = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]

    doy = np.sum(dayspermonth[0:month-1]) + day

    return doy


def dectime_to_timevec(dectime):
    # This subroutine converts dectime to individual hours, minutes and seconds

    doy = np.floor(dectime)

    DH = dectime-doy
    HOURS = int(24 * DH)

    DM=24*DH - HOURS
    MINS=int(60 * DM)

    DS = 60 * DM - MINS
    SECS = int(60 * DS)

    return (HOURS, MINS, SECS)

def saveraster(gdal_data, filename, raster):
    rows = gdal_data.RasterYSize
    cols = gdal_data.RasterXSize

    outDs = gdal.GetDriverByName("GTiff").Create(filename, cols, rows, int(1), GDT_Float32)
    outBand = outDs.GetRasterBand(1)

    # write the data
    outBand.WriteArray(raster, 0, 0)
    # flush data to disk, set the NoData value and calculate stats
    outBand.FlushCache()
    outBand.SetNoDataValue(-9999)

    # georeference the image and set the projection
    outDs.SetGeoTransform(gdal_data.GetGeoTransform())
    outDs.SetProjection(gdal_data.GetProjection())


In [7]:
shade_test = shadecalculation_setup(filepath_dsm = '../data/raw_data/R5_25GN1.TIF', date = dt.datetime.now(), intervalTime = 30, onetime = 0, filepath_save='../results/output/', UTC=0, dst=1)

2024-07-03 17:19:51.411304
PROJCS["Amersfoort / RD New",GEOGCS["Amersfoort",DATUM["Amersfoort",SPHEROID["Bessel 1841",6377397.155,299.1528128,AUTHORITY["EPSG","7004"]],AUTHORITY["EPSG","6289"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4289"]],PROJECTION["Oblique_Stereographic"],PARAMETER["latitude_of_origin",52.1561605555556],PARAMETER["central_meridian",5.38763888888889],PARAMETER["scale_factor",0.9999079],PARAMETER["false_easting",155000],PARAMETER["false_northing",463000],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","28992"]]


### FIRST TEST WITH SOLAR API GROUNDDSM AFTER FILTERING TREES ETC.

In [4]:
shade_test = shadecalculation_setup(filepath_dsm = '/Users/lbeuster/Documents/SAL/Projects/throwing_shade/data/clean_data/canopydsm/dsm_buildings_nonan.tif', date = dt.datetime.now(), intervalTime = 30, onetime = 0, filepath_save='../results/output/', UTC=2, dst=1)

2024-07-10 17:05:35.917623
PROJCS["WGS 84 / UTM zone 31N",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",3],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","32631"]]


/opt/anaconda3/envs/throwing_shade/lib/python3.12/site-packages/osgeo/gdal.py:312: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(
/var/folders/qg/ghhvk02j7t3f3b9th5d2412rxxy23f/T/ipykernel_29631/2470791893.py:294: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  alt[i] = 90. - sun['zenith']
/var/folders/qg/ghhvk02j7t3f3b9th5d2412rxxy23f/T/ipykernel_29631/2470791893.py:295: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  azi[i] = sun['azimuth']


## First test with vegfunction

In [3]:
shade_veg = shadecalculation_setup(filepath_dsm = '/Users/lbeuster/Documents/SAL/Projects/throwing_shade/data/clean_data/canopydsm/dsm_buildings_nonan.tif', filepath_veg = '/Users/lbeuster/Documents/SAL/Projects/throwing_shade/data/clean_data/canopydsm/canopy_dsm_nonan.tif', date = dt.datetime.now(), intervalTime = 30, onetime =0, filepath_save='../results/output/', UTC=0, dst=1, useveg=0, trunkheight=25)

2024-07-10 17:15:19.497710
PROJCS["WGS 84 / UTM zone 31N",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",3],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","32631"]]


/opt/anaconda3/envs/throwing_shade/lib/python3.12/site-packages/osgeo/gdal.py:312: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(
/var/folders/qg/ghhvk02j7t3f3b9th5d2412rxxy23f/T/ipykernel_31867/3325018280.py:294: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  alt[i] = 90. - sun['zenith']
/var/folders/qg/ghhvk02j7t3f3b9th5d2412rxxy23f/T/ipykernel_31867/3325018280.py:295: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  azi[i] = sun['azimuth']


## Stepwise processing of all dsm in folder:

In [22]:
# Create an empty dictionary to be filled with the landsat scenes
filenames = []
dsm_directory = '../../throwing_shade/data/clean_data/solar/2034624'
shade_directory = '../../throwing_shade/results/output/shade_cpt'

# Loop through all filenames in the directory
for filename in os.listdir(dsm_directory):
    # All raster files are stored as TIF, and since we only have B10, these are easy to pick out. Select these here. 
    if filename.endswith("dsm.tif"):
        filenames.append(filename)
        # for all files, open with open_rasterio. This needs the directory again. Saves the dataset in a dictionary which can be looped through later.
        # file_data[filename] = rioxarray.open_rasterio(f'../data/raw_data/Landsat/{filename}', masked=True).squeeze()
        continue
    else:
        continue

print(filenames)

sample_file = filenames[4]
print(sample_file)

full_filepath = dsm_directory + '/' + sample_file
print(full_filepath)

['2034624_p_3_2015_01_19_dsm.tif', '2034624_p_8_2015_01_19_dsm.tif', '2034624_p_5_2015_01_19_dsm.tif', '2034624_p_6_2015_01_19_dsm.tif', '2034624_p_7_2015_01_19_dsm.tif', '2034624_p_9_2015_01_19_dsm.tif', '2034624_p_4_2015_01_19_dsm.tif', '2034624_p_2_2015_01_19_dsm.tif', '2034624_p_1_2015_01_19_dsm.tif']
2034624_p_7_2015_01_19_dsm.tif
../../throwing_shade/data/clean_data/solar/2034624/2034624_p_7_2015_01_19_dsm.tif


In [23]:
%timeit
shade_calc = shadecalculation_setup(filepath_dsm=full_filepath, date = dt.datetime.now(), intervalTime = 30, onetime = 0, filepath_save = shade_directory)

2024-06-11 12:00:46.064118
PROJCS["WGS 84 / UTM zone 34S",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",21],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",10000000],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","32734"]]


# Tiling large rasters:

## Working, ish

In [31]:

def tile_raster(input_raster_path, output_folder, tile_size):
    dataset = gdal.Open(input_raster_path)
    band = dataset.GetRasterBand(1)
    width = dataset.RasterXSize
    height = dataset.RasterYSize
    tile_width, tile_height = tile_size

    def process_tile(tile_data):
        # Custom processing logic (e.g., calculate mean value)
        shade_test = shadecalculation_tiling(gdal_dsm=dataset, dsm = tile_data, filepath_dsm = input_raster_path, date = dt.datetime.now(), intervalTime = 30, onetime = 0, filepath_save=output_folder, UTC=0, dst=1)
        print('success')

    for i in range(0, width, tile_width):
        for j in range(0, height, tile_height):
            if i + tile_width > width:
                tile_width = width - i
            if j + tile_height > height:
                tile_height = height - j
            
            # Read the tile data
            tile_data = band.ReadAsArray(i, j, tile_width, tile_height).astype(float)
            
            # Process the tile data
            process_tile(tile_data)
            
            # # Save the tile (optional)
            # driver = gdal.GetDriverByName('GTiff')
            # output_filename = os.path.join(output_folder, f'tile_{i}_{j}.tif')
            # out_dataset = driver.Create(output_filename, tile_width, tile_height, 1, band.DataType)
            # out_dataset.GetRasterBand(1).WriteArray(tile_data)
            # out_dataset.SetGeoTransform((
            #     dataset.GetGeoTransform()[0] + i * dataset.GetGeoTransform()[1],
            #     dataset.GetGeoTransform()[1],
            #     0,
            #     dataset.GetGeoTransform()[3] + j * dataset.GetGeoTransform()[5],
            #     0,
            #     dataset.GetGeoTransform()[5]
            # ))
            # out_dataset.SetProjection(dataset.GetProjection())
            # out_dataset.FlushCache()

# Example usage
input_raster_path = '../data/raw_data/R5_25GN1.TIF'
output_folder = '../results/output/tiling/'
# Might have to make the tile size conditional to the total size (and always less than x million cells as described in the method)
tile_size = (256, 256)  # Adjust the tile size as needed

tile_raster(input_raster_path, output_folder, tile_size)

2024-05-31 17:39:34.063017
PROJCS["Amersfoort / RD New",GEOGCS["Amersfoort",DATUM["Amersfoort",SPHEROID["Bessel 1841",6377397.155,299.1528128,AUTHORITY["EPSG","7004"]],AUTHORITY["EPSG","6289"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4289"]],PROJECTION["Oblique_Stereographic"],PARAMETER["latitude_of_origin",52.1561605555556],PARAMETER["central_meridian",5.38763888888889],PARAMETER["scale_factor",0.9999079],PARAMETER["false_easting",155000],PARAMETER["false_northing",463000],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","28992"]]
success
2024-05-31 17:39:34.449690
PROJCS["Amersfoort / RD New",GEOGCS["Amersfoort",DATUM["Amersfoort",SPHEROID["Bessel 1841",6377397.155,299.1528128,AUTHORITY["EPSG","7004"]],AUTHORITY["EPSG","6289"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG",

In [53]:

def calculate_tile_size(width, height, max_cells_per_tile):
    total_cells = width * height
    num_tiles = np.ceil(total_cells / max_cells_per_tile)

    # Find the closest integer dimensions that meet the criteria
    tile_width = int(np.ceil(np.sqrt(max_cells_per_tile * width / height)))
    tile_height = int(np.ceil(max_cells_per_tile / tile_width))

    return tile_width, tile_height


def tile_raster(input_raster_path, output_folder, max_cells_per_tile=4000000):
    dataset = gdal.Open(input_raster_path)
    band = dataset.GetRasterBand(1)
    width = dataset.RasterXSize
    height = dataset.RasterYSize


    tile_width, tile_height = calculate_tile_size(width, height, max_cells_per_tile)
    print(f'width: {tile_width}, height:  {tile_height}')

    counter = 1  # Initialize tile counter


    def process_tile(tile_data, counter):
        # Custom processing logic (e.g., calculate mean value)
        shade_test = shadecalculation_tiling(gdal_dsm=dataset, dsm = tile_data, filepath_dsm = input_raster_path, date = dt.datetime.now(), intervalTime = 30, onetime = 0, filepath_save=output_folder, UTC=0, dst=1, counter=counter)
        print('success')

    for i in range(0, width, tile_width):
            for j in range(0, height, tile_height):
                current_tile_width = min(tile_width, width - i)
                current_tile_height = min(tile_height, height - j)
            
            # Read the tile data
            tile_data = band.ReadAsArray(i, j, current_tile_width, current_tile_height)

            counter += 1
            
            # Process the tile data
            process_tile(tile_data, counter)
            
            # # Save the tile (optional)
            # driver = gdal.GetDriverByName('GTiff')
            # output_filename = os.path.join(output_folder, f'tile_{i}_{j}.tif')
            # out_dataset = driver.Create(output_filename, tile_width, tile_height, 1, band.DataType)
            # out_dataset.GetRasterBand(1).WriteArray(tile_data)
            # out_dataset.SetGeoTransform((
            #     dataset.GetGeoTransform()[0] + i * dataset.GetGeoTransform()[1],
            #     dataset.GetGeoTransform()[1],
            #     0,
            #     dataset.GetGeoTransform()[3] + j * dataset.GetGeoTransform()[5],
            #     0,
            #     dataset.GetGeoTransform()[5]
            # ))
            # out_dataset.SetProjection(dataset.GetProjection())
            # out_dataset.FlushCache()

# Example usage
input_raster_path = '../data/raw_data/R5_25GN1.TIF'
output_folder = '../results/output/tiling/'
# Might have to make the tile size conditional to the total size (and always less than x million cells as described in the method)
max_cells_per_tile = 50000  # Define the maximum number of cells per tile

tile_raster(input_raster_path, output_folder, max_cells_per_tile)

width: 200, height:  250
2024-05-31 18:20:47.028215
success
2024-05-31 18:20:47.424525
success
2024-05-31 18:20:47.666856
success
2024-05-31 18:20:47.815514
success
2024-05-31 18:20:48.030624
success


In [55]:
# Example usage
input_raster_path = '../data/raw_data/R_25GN1.TIF'
output_folder = '../results/output/tiling/'
# Might have to make the tile size conditional to the total size (and always less than x million cells as described in the method)
max_cells_per_tile = 4000000  # Define the maximum number of cells per tile

tile_raster(input_raster_path, output_folder, max_cells_per_tile)

width: 1789, height:  2236
2024-06-01 08:52:58.357794
success
2024-06-01 08:53:57.035617
success
2024-06-01 08:54:31.559255
success
2024-06-01 08:54:59.707813
success
2024-06-01 08:55:37.593442
success
2024-06-01 08:56:12.730997
success


In [52]:
def shadecalculation_tiling(gdal_dsm, dsm, filepath_dsm = 'None', date = dt.datetime.now(), intervalTime = 30, onetime =1, filepath_save='None', UTC=0, dst=1, counter=0):
    '''Calculates spot, hourly and or daily shading for a DSM
    Needs: 
    filepath_dsm = a path to a (building) dsm,
    date = a datetime object (defaults to datetime.now()),
    intervaltime = a integer in minutes as interval,
    onetime = to differentiate between single shade cast and day (default is single timestamp)
    filepath_save = 'path to the folder in which to save the files'
    UTC = defaults to 0, optional in plugin
    dst = defaults to 1, not sure what this does. 1 or 0
    '''
    print(date)


    # gdal_dsm = gdal.Open(filepath_dsm)
    
    # dsm = gdal_dsm.ReadAsArray().astype(float)

    # response to issue #85
    nd = gdal_dsm.GetRasterBand(1).GetNoDataValue()
    dsm[dsm == nd] = 0.
    if dsm.min() < 0:
        dsm = dsm + np.abs(dsm.min())

    sizex = dsm.shape[0]
    sizey = dsm.shape[1]

    old_cs = osr.SpatialReference()
    dsm_ref = gdal_dsm.GetProjection()
    # print(dsm_ref)
    # dsm_ref = dsmlayer.crs().toWkt()
    old_cs.ImportFromWkt(dsm_ref)

    wgs84_wkt = """
    GEOGCS["WGS 84",
        DATUM["WGS_1984",
            SPHEROID["WGS 84",6378137,298.257223563,
                AUTHORITY["EPSG","7030"]],
            AUTHORITY["EPSG","6326"]],
        PRIMEM["Greenwich",0,
            AUTHORITY["EPSG","8901"]],
        UNIT["degree",0.01745329251994328,
            AUTHORITY["EPSG","9122"]],
        AUTHORITY["EPSG","4326"]]"""

    new_cs = osr.SpatialReference()
    new_cs.ImportFromWkt(wgs84_wkt)

    transform = osr.CoordinateTransformation(old_cs, new_cs)

    width = gdal_dsm.RasterXSize
    height = gdal_dsm.RasterYSize
    gt = gdal_dsm.GetGeoTransform()
    minx = gt[0]
    miny = gt[3] + width*gt[4] + height*gt[5]
    lonlat = transform.TransformPoint(minx, miny)
    geotransform = gdal_dsm.GetGeoTransform()
    scale = 1 / geotransform[1]
    
    gdalver = float(gdal.__version__[0])
    if gdalver >= 3.:
        lon = lonlat[1] #changed to gdal 3
        lat = lonlat[0] #changed to gdal 3
    else:
        lon = lonlat[0] #changed to gdal 2
        lat = lonlat[1] #changed to gdal 2
    # print('lon:' + str(lon))
    # print('lat:' + str(lat))

    
    if filepath_dsm == 'None':
        print("Error", "No dsm path given")
        return
    else:
        # date = self.dlg.calendarWidget.selectedDate()
        year = date.year
        month = date.month
        day = date.day
        # TODO: Check what UTC does. Optional in plugin, so set to 0
        # UTC = self.dlg.spinBoxUTC.value()
        if onetime == 1:
            time = date.time()
            print(time)
            hour = time.hour
            minu = time.minute
            sec = time.second
        else:
            onetime = 0
            hour = 0
            minu = 0
            sec = 0

        tv = [year, month, day, hour, minu, sec]
        # TODO: What exactly is set by timeInterval.
        # intervalTime = self.dlg.intervalTimeEdit.time()
        # self.timeInterval = intervalTime.minute() + (intervalTime.hour() * 60) + (intervalTime.second()/60)

        shadowresult = dailyshading_tiling(dsm, scale, lon, lat, sizex, sizey, tv, UTC,
                                    intervalTime, onetime, filepath_save, gdal_dsm, 
                                    dst, counter)

        shfinal = shadowresult["shfinal"]
        time_vector = shadowresult["time_vector"]

        if onetime == 0:
            timestr = time_vector.strftime("%Y%m%d")
            savestr = '/shadow_fraction_on_'
        else:
            timestr = time_vector.strftime("%Y%m%d_%H%M")
            savestr = '/Shadow_at_'

    filename = filepath_save + savestr + timestr + '_tile_' + str(counter) + '.tif'

    saveraster_tiling(gdal_dsm, filename, shfinal)

def dailyshading_tiling(dsm, scale, lon, lat, sizex, sizey, tv, UTC, timeInterval, onetime, folder, gdal_data, dst, counter):

    # lon = lonlat[0]
    # lat = lonlat[1]
    year = tv[0]
    month = tv[1]
    day = tv[2]

    alt = np.median(dsm)
    location = {'longitude': lon, 'latitude': lat, 'altitude': alt}

    ## Taken out of the usevegdem if, main shadecasting loop depends on it. 
    # if usevegdem == 1:
    #     psi = trans
    #     # amaxvalue
    #     vegmax = vegdsm.max()
    #     amaxvalue = dsm.max() - dsm.min()
    #     amaxvalue = np.maximum(amaxvalue, vegmax)

    #     # Elevation vegdsms if buildingDSM includes ground heights
    #     vegdem = vegdsm + dsm
    #     vegdem[vegdem == dsm] = 0
    #     vegdem2 = vegdsm2 + dsm
    #     vegdem2[vegdem2 == dsm] = 0

    #     # Bush separation
    #     bush = np.logical_not((vegdem2*vegdem))*vegdem

    # #     vegshtot = np.zeros((sizex, sizey))
    # # else:
        
    shtot = np.zeros((sizex, sizey))

    if onetime == 1:
        itera = 1
    else:
        itera = int(np.round(1440 / timeInterval))

    alt = np.zeros(itera)
    azi = np.zeros(itera)
    hour = int(0)
    index = 0
    time = dict()
    time['UTC'] = UTC

    # if wallshadow == 1:
    #     walls = wheight
    #     dirwalls = waspect
    # else: 
    walls = np.zeros((sizex, sizey))
    dirwalls = np.zeros((sizex, sizey))

    for i in range(0, itera):
        if onetime == 0:
            minu = int(timeInterval * i)
            if minu >= 60:
                hour = int(np.floor(minu / 60))
                minu = int(minu - hour * 60)
        else:
            minu = tv[4]
            hour = tv[3]
        
        doy = day_of_year(year, month, day)

        ut_time = doy - 1. + ((hour - dst) / 24.0) + (minu / (60. * 24.0)) + (0. / (60. * 60. * 24.0))
        
        if ut_time < 0:
            year = year - 1
            month = 12
            day = 31
            doy = day_of_year(year, month, day)
            ut_time = ut_time + doy - 1

        HHMMSS = dectime_to_timevec(ut_time)
        
        time['year'] = year
        time['month'] = month
        time['day'] = day
        time['hour'] = HHMMSS[0]
        time['min'] = HHMMSS[1]
        time['sec'] = HHMMSS[2]

        sun = sp.sun_position(time, location)
        alt[i] = 90. - sun['zenith']
        azi[i] = sun['azimuth']

        if time['sec'] == 59: #issue 228 and 256
            time['sec'] = 0
            time['min'] = time['min'] + 1
            if time['min'] == 60:
                time['min'] = 0
                time['hour'] = time['hour'] + 1
                if time['hour'] == 24:
                    time['hour'] = 0

        time_vector = dt.datetime(year, month, day, time['hour'], time['min'], time['sec'])
        timestr = time_vector.strftime("%Y%m%d_%H%M")

        if alt[i] > 0:
            # if wallshadow == 1: # Include wall shadows (Issue #121)
            #     if usevegdem == 1:
            #         vegsh, sh, _, wallsh, _, wallshve, _, _ = shadowingfunction_wallheight_23(dsm, vegdem, vegdem2,
            #                                     azi[i], alt[i], scale, amaxvalue, bush, walls, dirwalls * np.pi / 180.)
            #         sh = sh - (1 - vegsh) * (1 - psi)
            #         if onetime == 0:
            #             filenamewallshve = folder + '/Facadeshadow_fromvegetation_' + timestr + '_LST.tif'
            #             saveraster(gdal_data, filenamewallshve, wallshve)
            #     else:
            #         sh, wallsh, _, _, _ = shadowingfunction_wallheight_13(dsm, azi[i], alt[i], scale,
            #                                                                             walls, dirwalls * np.pi / 180.)
            #         # shtot = shtot + sh
                
            #     if onetime == 0:
            #         filename = folder + '/Shadow_ground_' + timestr + '_LST.tif'
            #         saveraster(gdal_data, filename, sh)
            #         filenamewallsh = folder + '/Facadeshadow_frombuilding_' + timestr + '_LST.tif'
            #         saveraster(gdal_data, filenamewallsh, wallsh)
                    

            # else:
            #     if usevegdem == 0:
            #         sh = shadow.shadowingfunctionglobalradiation(dsm, azi[i], alt[i], scale, dlg, 0)
            #         # shtot = shtot + sh
            #     else:
            # shadowresult = shadow.shadowingfunction_20(dsm, azi[i], alt[i], scale, amaxvalue, 0)


            # vegsh = shadowresult["vegsh"]
            # sh = shadowresult["sh"]
            # sh=sh-(1-vegsh)*(1-psi)
            # vegshtot = vegshtot + sh
            sh = shadow.shadowingfunctionglobalradiation(dsm, azi[i], alt[i], scale, 0)

            if onetime == 0:
                filename = folder + '/Shadow_' + timestr + '_tile_' + str(counter) + '_LST.tif'
                ## EDITED 
                saveraster_tiling(gdal_data,filename, sh)

                shtot = shtot + sh
                index += 1

    shfinal = shtot / index

    # if wallshadow == 1:
    #     if onetime == 1:
    #         filenamewallsh = folder + '/Facadeshadow_frombuilding_' + timestr + '_LST.tif'
    #         saveraster(gdal_data, filenamewallsh, wallsh)
    #         if usevegdem == 1:
    #             filenamewallshve = folder + '/Facadeshadow_fromvegetation_' + timestr + '_LST.tif'
    #             saveraster(gdal_data, filenamewallshve, wallshve)

    shadowresult = {'shfinal': shfinal, 'time_vector': time_vector}

    # dlg.progressBar.setValue(0)

    return shadowresult



def saveraster_tiling(gdal_data, filename, raster):
    dimensions = raster.shape
    rows, cols = dimensions

    outDs = gdal.GetDriverByName("GTiff").Create(filename, cols, rows, int(1), GDT_Float32)
    outBand = outDs.GetRasterBand(1)

    # write the data
    outBand.WriteArray(raster, 0, 0)
    # flush data to disk, set the NoData value and calculate stats
    outBand.FlushCache()
    outBand.SetNoDataValue(-9999)

    # georeference the image and set the projection
    outDs.SetGeoTransform(gdal_data.GetGeoTransform())
    outDs.SetProjection(gdal_data.GetProjection())